In [ ]:
# MUST READ: This is for all three bands (g, r, i)
# stretch: scaling constant; we can think of it as brightness. 
#small stretch -> brighter, but bring out noise; big stretch -> darker
# Q: how strongly bright pixels are “softened” relative to the faint one
#high Q -> more compression of bright regions; lower Q -> more linear behavior

In [ ]:
from lsst.daf.butler import Butler
import lsst.afw.display as afw_display
import lsst.geom as geom
from lsst.afw.image import MultibandExposure
from lsst.pipe.tasks.prettyPictureMaker import PrettyPictureConfig, PrettyPictureTask
from lsst.pipe.tasks.prettyPictureMaker._task import ChannelRGBConfig

import pandas as pd
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import make_lupton_rgb
from astropy.table import Table, Column
import matplotlib.pyplot as plt
from astropy.table import Table
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

afw_display.setDefaultBackend('matplotlib')

In [ ]:
butler = Butler("dp1", collections="LSSTComCam/DP1")
assert butler is not None
skymap = butler.get("skyMap")

In [ ]:
def showRGB(image, bgr="gri", ax=None, fp=None, figsize=(8,8), stretch=100, Q=1, name=None):
    """Display an RGB color composite image with matplotlib.
    
    Parameters
    ----------
    image : `MultibandImage`
        `MultibandImage` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e. keys of the exps dict) indicating what band
        to use for each channel. If `image` only has three filters then this parameter is ignored
        and the filters in the image are used.
    ax : `matplotlib.axes.Axes`
        Axis in a `matplotlib.Figure` to display the image.
        If `axis` is `None` then a new figure is created.
    figsize: tuple
        Size of the `matplotlib.Figure` created.
        If `ax` is not `None` then this parameter is ignored.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    name: str
        The name of the object/field to be displayed.
    """
    # If the image only has 3 bands, reverse the order of the bands to produce the RGB image
    if len(image) == 3:
        bgr = image.filters
    # Extract the primary image component of each Exposure with the .image property, and use .array to get a NumPy array view.
    rgb = make_lupton_rgb(image_r=image[bgr[2]].array,  # numpy array for the r channel
                          image_g=image[bgr[1]].array,  # numpy array for the g channel
                          image_b=image[bgr[0]].array,  # numpy array for the b channel
                          stretch=stretch, Q=Q)  # parameters used to stretch and scale the pixel values
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1,1,1)
    
    plt.axis("off")
    ax.imshow(rgb, interpolation='nearest', origin='lower')

    if name is not None:
        plt.text(0, SIZE_PIX-1, name, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
    
    plt.text(0, 2, 'astropy Lupton '+bgr, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
    
    plt.tight_layout();

    return

In [ ]:
def prettyRGB(images, ax=None, fp=None, figsize=(8,8), stretch=250, Q=0.7, name=None):
    """Display an RGB (irg) color composite image with matplotlib using the prettyPictureMaker pipe task.
    
    Parameters
    ----------
    images : dict
        Dictionary of images to display.
    ax : `matplotlib.axes.Axes`
        Axis in a `matplotlib.Figure` to display the image.
        If `axis` is `None` then a new figure is created.
    figsize: tuple
        Size of the `matplotlib.Figure` created.
        If `ax` is not `None` then this parameter is ignored.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    name: str
        The name of the object/field to be displayed.
    """

    prettyPicConfig = PrettyPictureTask.ConfigClass()
    # Magic from Nate Lust:
    prettyPicConfig.localContrastConfig.doLocalContrast = False
    prettyPicConfig.localContrastConfig.sigma = 30
    prettyPicConfig.localContrastConfig.clarity = 0.8
    prettyPicConfig.localContrastConfig.shadows = 0
    prettyPicConfig.localContrastConfig.highlights = -1.5
    prettyPicConfig.localContrastConfig.maxLevel = 2
    prettyPicConfig.imageRemappingConfig.absMax = 11000
    prettyPicConfig.luminanceConfig.max = 100
    prettyPicConfig.luminanceConfig.stretch = stretch    # from kwargs
    prettyPicConfig.luminanceConfig.floor = 0
    prettyPicConfig.luminanceConfig.Q = Q                # from kwargs
    prettyPicConfig.luminanceConfig.highlight = 0.905882
    prettyPicConfig.luminanceConfig.shadow = 0.12
    prettyPicConfig.luminanceConfig.midtone = 0.25
    prettyPicConfig.doPSFDeconcovlve = False   # sic
    prettyPicConfig.exposureBrackets = None
    prettyPicConfig.colorConfig.maxChroma = 80
    prettyPicConfig.colorConfig.saturation = 0.6
    prettyPicConfig.cieWhitePoint = (0.28, 0.28)
    prettyPicConfig.channelConfig = dict(
        g=ChannelRGBConfig(r=0.0, g=0.0, b=1.0),
        r=ChannelRGBConfig(r=0.0, g=1.0, b=0.0),
        i=ChannelRGBConfig(r=1.0, g=0.0, b=0.0),
    )
    prettyPicTask = PrettyPictureTask(config=prettyPicConfig)
    
    bands = "gri"
    coaddG = images['g']
    coaddR = images['r']
    coaddI = images['i']
    
    prettyPicInputs = prettyPicTask.makeInputsFromExposures(i=coaddI, r=coaddR, g=coaddG)
    coaddRgbStruct = prettyPicTask.run(prettyPicInputs)
    coaddRgb = coaddRgbStruct.outputRGB

    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1,1,1)
    
    plt.axis("off")
    ax.imshow(coaddRgb, interpolation='nearest', origin='lower')

    if name is not None:
        plt.text(0, SIZE_PIX-1, name, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
    
    plt.text(0, 2, 'PrettyPictureTask '+bands, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
    
    plt.tight_layout();

    return

In [ ]:
def init_worker(repo="dp1", collection="LSSTComCam/DP1"):
    global BUTLER, SKYMAP
    BUTLER = Butler(repo, collections=collection)
    SKYMAP = BUTLER.get("skyMap")

def process_one(name, ra, dec):
    import matplotlib
    matplotlib.use("Agg") #non-interactive backend for multi-processing & image available
    import matplotlib.pyplot as plt
    import lsst.geom as geom

    try:
        radec = geom.SpherePoint(float(ra), float(dec), geom.degrees)
        tractInfo = SKYMAP.findTract(radec)
        patchInfo = tractInfo.findPatch(radec)
        tract = tractInfo.getId()
        patch = tractInfo.getSequentialPatchIndex(patchInfo)

        xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
        bbox = geom.BoxI(xy - geom.ExtentI(SIZE_PIX, SIZE_PIX)//2, geom.ExtentI(SIZE_PIX, SIZE_PIX))
        params = {"bbox": bbox}

        per_band = {}
        for b in BANDS:
            dataId = {"tract": tract, "patch": patch, "band": b}
            per_band[b] = BUTLER.get("deep_coadd", parameters=params, dataId=dataId)

        fig, ax = plt.subplots(1, 1, figsize=(3, 3))
        prettyRGB(per_band, ax=ax, stretch=STRETCH, Q=Q_SOFT, name=name)
        png_path = os.path.join(OUT_PNG, f"{name}_gri_pretty.png")
        fig.savefig(png_path, dpi=150, bbox_inches="tight")
        plt.close(fig)

        return {"name": name, "ok": True, "png": png_path}
    except Exception as e:
        return {"name": name, "ok": False, "error": str(e)}


In [ ]:
df = pd.read_csv("candels_hst.csv", usecols=["Shortname", "RA", "DEC", "i"])
df["RA"] = df["RA"].astype(float)
df["DEC"] = df["DEC"].astype(float)

In [ ]:
BANDS = ("g","r","i")
SIZE_PIX = 40
MAX_WORKERS = min(4, os.cpu_count())
OUT_PNG = "goods_png_gri"
STRETCH = 400 #750 #scaling constant
Q_SOFT = 1.5 #0.7 #core looks blown out: need to increase Q  

#os.makedirs(OUT_PNG, exist_ok=True) #for creating the folder

BUTLER = None
SKYMAP = None

In [ ]:
# Only run the first 3 rows to test
#TEST_N = 25
#df_test = df.head(TEST_N).copy()

results = []
with ProcessPoolExecutor(max_workers=MAX_WORKERS, initializer=init_worker) as ex:
    futures = [ex.submit(process_one, row["Shortname"], row["RA"], row["DEC"]) for _, row in df.iterrows()]
    for fut in as_completed(futures):
        results.append(fut.result())

ok = sum(1 for r in results if r["ok"])
fail = len(results) - ok
print(f"[FULL RUN] Success: {ok}, Failed: {fail}")
if fail:
    for r in results:
        if not r["ok"]:
            print("  ", r["name"], "->", r["error"])
